Importing the Python Libraries needed to process the data.

In [1]:
import pandas as pd
import numpy as np
import re
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
print("Library Import Complete")

Library Import Complete


Importing the CISC-2010 Dataset stored in the IBM asset.

In [2]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_5152ce06f6254e56af8f05b5b805e7a9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='')

body = client_5152ce06f6254e56af8f05b5b805e7a9.get_object(Bucket='',Key='input_file.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1 = df_data_1.replace(np.nan, '', regex=True)
df_data_1.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,index,method,url,protocol,userAgent,pragma,cacheControl,accept,acceptEncoding,acceptCharset,acceptLanguage,host,connection,contentLength,contentType,cookie,payload,label
0,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,,,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,id=2,anom
1,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,,,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,nombre=Jam%F3n+Ib%E9rico,anom
2,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,,,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,precio=85,anom
3,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,,,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,cantidad=%27%3B+DROP+TABLE+usuarios%3B+SELECT+...,anom
4,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,,,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,B1=A%F1adir+al+carrito,anom


Processing the Data for the following features:
1. Length of request ....


In [3]:
def count_alpha_numeric_spchars(string):
    """return the number of digits, characters and special characters present in a string"""
    string = str(string)
    list = [0,0,0] # digit, char, sp_chars
    string_check= re.compile('[@_!#$%^&*()<>?/\|}{~:]')
    for ch in string:
        if ch.isdigit():
            list[0] += 1
        elif (string_check.search(ch) != None):
            list[2] += 1
        else:
            list[1] += 1
    return list[0], list[1], list[2]

In [4]:
print("The number of digits, characters and special characters present in the string are : ",count_alpha_numeric_spchars("Helloe123@!#"))

The number of digits, characters and special characters present in the string are :  (3, 6, 3)


In [5]:
def path(url):
    return "/".join(url.split('/')[3::])

In [6]:
print(path("http://localhost:8080/tienda1/publico/anadir.jsp"))

tienda1/publico/anadir.jsp


In [7]:
def label_fix(string):
    l = string.split(',')
    if 'anom' in l:
        return 0
    else:
        return 1

In [8]:
df_data_1['length_payload'] = df_data_1["payload"].str.len()
df_data_1['nd_payload'], df_data_1['nc_payload'], df_data_1['nsp_payload'] =  zip(*df_data_1['payload'].map(count_alpha_numeric_spchars))
df_data_1['trucated_url'] = df_data_1['url'].apply(path)
df_data_1['nd_url'], df_data_1['nc_url'], df_data_1['nsp_url'] =  zip(*df_data_1['trucated_url'].map(count_alpha_numeric_spchars))
df_data_1['length_url'] = df_data_1['trucated_url'].str.len()

df_data_1.head()

,index,method,url,protocol,userAgent,pragma,cacheControl,accept,acceptEncoding,acceptCharset,...,label,length_payload,nd_payload,nc_payload,nsp_payload,trucated_url,nd_url,nc_url,nsp_url,length_url
0,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",...,anom,4,1,3,0,tienda1/publico/anadir.jsp,1,23,2,26
1,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",...,anom,24,2,20,2,tienda1/publico/anadir.jsp,1,23,2,26
2,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",...,anom,9,2,7,0,tienda1/publico/anadir.jsp,1,23,2,26
3,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",...,anom,83,8,69,6,tienda1/publico/anadir.jsp,1,23,2,26
4,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",...,anom,22,2,19,1,tienda1/publico/anadir.jsp,1,23,2,26


In [9]:
#performing group-by operation 
grouped_single = df_data_1.groupby('index').agg({'length_payload': ['sum'],
                                                'nd_payload': ['sum'],
                                                'nc_payload': ['sum'],
                                                'nsp_payload': ['sum'],
                                                'nd_url': ['sum'],
                                                'nc_url': ['sum'],
                                                'nsp_url': ['sum'],
                                                'length_url': ['max'], 
                                                'method' : ['max']
                                                })


grouped_single['count_args'] = df_data_1.groupby("index")["payload"].size()
grouped_single['label_join'] = df_data_1.groupby("index")["label"].apply(lambda label: ','.join(label))

grouped_single['label'] = grouped_single['label_join'].apply(label_fix)
grouped_single.head()

,length_payload,nd_payload,nc_payload,nsp_payload,nd_url,nc_url,nsp_url,length_url,method,count_args,label_join,label
,sum,sum,sum,sum,sum,sum,sum,max,max,,,
index,,,,,,,,,,,,
0,142,15,118,9,6,130,11,26,GET,6,"anom,anom,anom,anom,anom,norm",0
1,137,18,114,5,10,230,20,26,GET,10,"anom,anom,anom,anom,anom,norm,norm,norm,norm,norm",0
2,59,4,54,1,5,152,11,30,GET,6,"anom,norm,norm,norm,norm,norm",0
3,118,19,91,8,6,167,12,35,GET,6,"anom,anom,anom,anom,anom,norm",0
4,65,4,58,3,6,159,12,30,GET,6,"anom,anom,anom,anom,anom,norm",0


In [10]:
def encode_method(string):
    if string == 'GET':
        return 1, 0, 0
    elif string == 'POST':
        return 0, 1, 0
    else:
        return 0, 0, 1

In [11]:
encode_method('GET')

(1, 0, 0)

In [12]:
grouped_single['method_get'], grouped_single['method_post'], grouped_single['method_put'] = zip(*grouped_single['method']['max'].map(encode_method))
grouped_single.head()

,length_payload,nd_payload,nc_payload,nsp_payload,nd_url,nc_url,nsp_url,length_url,method,count_args,label_join,label,method_get,method_post,method_put
,sum,sum,sum,sum,sum,sum,sum,max,max,,,,,,
index,,,,,,,,,,,,,,,
0,142,15,118,9,6,130,11,26,GET,6,"anom,anom,anom,anom,anom,norm",0,1,0,0
1,137,18,114,5,10,230,20,26,GET,10,"anom,anom,anom,anom,anom,norm,norm,norm,norm,norm",0,1,0,0
2,59,4,54,1,5,152,11,30,GET,6,"anom,norm,norm,norm,norm,norm",0,1,0,0
3,118,19,91,8,6,167,12,35,GET,6,"anom,anom,anom,anom,anom,norm",0,1,0,0
4,65,4,58,3,6,159,12,30,GET,6,"anom,anom,anom,anom,anom,norm",0,1,0,0


In [30]:
df = pd.DataFrame(columns = ['method_get', 'method_post', 'method_put', 'length_request', 'length_args', 'number_args', 'number_digits', 'length_path', 'number_char', 'number_spchar', 'label'])
df['method_get'] = grouped_single['method_get']
df['method_post'] = grouped_single['method_post']
df['method_put'] = grouped_single['method_put']
df['length_request'] =   grouped_single['length_payload']['sum'] +grouped_single['length_url']['max'] 
df['length_args'] = grouped_single['length_payload']['sum']
df['number_args'] = grouped_single['count_args']
df['number_digits'] =  grouped_single['nd_payload']['sum'] #grouped_single['nd_url']['sum']  
df['length_path'] = grouped_single['length_url']['max']
df['number_char'] = grouped_single['nc_payload']['sum'] #+ grouped_single['nc_url']['sum']
df['number_spchar'] = grouped_single['nsp_payload']['sum'] +grouped_single['nsp_url']['sum']  
df['label'] = grouped_single['label']
df.head()

,method_get,method_post,method_put,length_request,length_args,number_args,number_digits,length_path,number_char,number_spchar,label
index,,,,,,,,,,,
0,1,0,0,168,142,6,15,26,118,20,0
1,1,0,0,163,137,10,18,26,114,25,0
2,1,0,0,89,59,6,4,30,54,12,0
3,1,0,0,153,118,6,19,35,91,20,0
4,1,0,0,95,65,6,4,30,58,15,0


Pre-processing completed ...

In [47]:
#assigning inputs and outputs
X = df.iloc[:, [0,1,2,3,4,5,6,7,8,9]].values
y = df.iloc[:, -1].values

In [44]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
# Training the Naive Bayes model on the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [45]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 819 5425]
 [   1 2755]]


In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.39711111111111114